In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import h5py
import pandas as pd
import seaborn as sns
from pandas import DataFrame

import tensorflow as tf
from tensorflow.keras import layers, models
import random

# Set a specific random seed
random.seed(42)

In [3]:
import torch
torch.cuda.is_available()
torch.cuda.get_device_name(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = torch.tensor([1.0, 2.0, 3.0])
x = x.to(device)
print(x.device)
torch.cuda.get_device_name(0)

cuda:0


'NVIDIA GeForce RTX 2060'

In [4]:
# Time tracking, Operation time (min):  0.003
filename="N-CMAPSS_DS02-006.h5"
t = time.process_time()  

# Load data
with h5py.File(filename, 'r') as hdf:
        # Development set
        W_dev = np.array(hdf.get('W_dev'))             # W
        X_s_dev = np.array(hdf.get('X_s_dev'))         # X_s
        X_v_dev = np.array(hdf.get('X_v_dev'))         # X_v
        T_dev = np.array(hdf.get('T_dev'))             # T
        Y_dev = np.array(hdf.get('Y_dev'))             # RUL  
        A_dev = np.array(hdf.get('A_dev'))             # Auxiliary

        # Test set
        W_test = np.array(hdf.get('W_test'))           # W
        X_s_test = np.array(hdf.get('X_s_test'))       # X_s
        X_v_test = np.array(hdf.get('X_v_test'))       # X_v
        T_test = np.array(hdf.get('T_test'))           # T
        Y_test = np.array(hdf.get('Y_test'))           # RUL  
        A_test = np.array(hdf.get('A_test'))           # Auxiliary
        
        # Varnams
        W_var = np.array(hdf.get('W_var'))
        X_s_var = np.array(hdf.get('X_s_var'))  
        X_v_var = np.array(hdf.get('X_v_var')) 
        T_var = np.array(hdf.get('T_var'))
        A_var = np.array(hdf.get('A_var'))
        
        # from np.array to list dtype U4/U5
        W_var = list(np.array(W_var, dtype='U20'))
        X_s_var = list(np.array(X_s_var, dtype='U20'))  
        X_v_var = list(np.array(X_v_var, dtype='U20')) 
        T_var = list(np.array(T_var, dtype='U20'))
        A_var = list(np.array(A_var, dtype='U20'))
                          
W = np.concatenate((W_dev, W_test), axis=0)  
X_s = np.concatenate((X_s_dev, X_s_test), axis=0)
X_v = np.concatenate((X_v_dev, X_v_test), axis=0)
T = np.concatenate((T_dev, T_test), axis=0)
Y = np.concatenate((Y_dev, Y_test), axis=0) 
A = np.concatenate((A_dev, A_test), axis=0) 
    
print('')
print("Operation time (min): " , (time.process_time()-t)/60)
print('')
print ("W shape: " + str(W.shape))
print ("X_s shape: " + str(X_s.shape))
print ("X_v shape: " + str(X_v.shape))
print ("T shape: " + str(T.shape))
print ("A shape: " + str(A.shape))


Operation time (min):  0.043229166666666666

W shape: (6517190, 4)
X_s shape: (6517190, 14)
X_v shape: (6517190, 14)
T shape: (6517190, 10)
A shape: (6517190, 4)


In [5]:
df_W = DataFrame(data=W, columns=W_var)
df_X_s_parameters = DataFrame(data=X_s, columns=X_s_var)
df_X_v_parameters = DataFrame(data=X_v, columns=X_v_var)
df_T = DataFrame(data=T, columns=T_var)
df_Y = DataFrame(data=Y, columns=['RUL'])
df_A = DataFrame(data=A, columns=A_var)
df_all_units=pd.concat([df_W,df_A,df_X_s_parameters,df_X_v_parameters,df_T,df_Y],axis=1)
def prepare_data(dataframe,list_of_columns_to_drop):
    dataframe=dataframe.drop(columns=list_of_columns_to_drop)
    return dataframe
def data_test_train_splitter(DataFrame,train_idx,test_idx,list_of_columns_to_drop):
    train_df=prepare_data(DataFrame.loc[DataFrame["unit"].isin(train_idx)],list_of_columns_to_drop)
    test_df=prepare_data(DataFrame.loc[DataFrame["unit"].isin(test_idx)],list_of_columns_to_drop)
    return (train_df,test_df)

In [39]:
train_idx=[16]
test_idx=[14,11,15]
# list_of_columns_to_drop=["alt","Mach","TRA",'fan_eff_mod','fan_flow_mod','LPC_eff_mod','HPC_eff_mod','HPC_flow_mod','HPT_flow_mod','LPT_eff_mod','LPT_flow_mod']

list_of_columns_to_drop=["alt","Mach","TRA","fan_eff_mod","fan_flow_mod","LPC_eff_mod","LPC_flow_mod","HPC_flow_mod","HPC_eff_mod","HPT_flow_mod"]
train,test=data_test_train_splitter(df_all_units,train_idx,test_idx,list_of_columns_to_drop)
# train.drop(columns=["unit","Fc"])
# test.drop(columns=["unit","Fc"])
train  


,T2,unit,cycle,Fc,hs,T24,T30,T48,T50,P15,...,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod,RUL
2839273,503.721197,16.0,1.0,3.0,1.0,602.809026,1445.734091,1831.247582,1234.668125,16.004249,...,219.132512,231.882086,16.755149,9.826825,25.205333,42.167109,-0.000748,-0.000499,-0.000470,62
2839274,503.699197,16.0,1.0,3.0,1.0,602.884699,1445.729730,1830.328855,1233.775609,16.004450,...,219.145580,231.898897,16.747632,9.812432,25.243954,42.128645,-0.000748,-0.000499,-0.000470,62
2839275,503.682048,16.0,1.0,3.0,1.0,602.850026,1445.623942,1830.356293,1233.817761,15.999599,...,219.061447,231.810175,16.751288,9.804538,25.239399,42.133718,-0.000748,-0.000499,-0.000470,62
2839276,503.653483,16.0,1.0,3.0,1.0,602.817989,1445.560836,1830.231656,1233.717364,15.995012,...,219.009495,231.755178,16.751436,9.806258,25.240772,42.130588,-0.000748,-0.000499,-0.000470,62
2839277,503.645699,16.0,1.0,3.0,1.0,602.803554,1445.513032,1830.195152,1233.683058,15.992129,...,218.963946,231.706993,16.754685,9.804464,25.240621,42.130812,-0.000748,-0.000499,-0.000470,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3604563,502.759104,16.0,63.0,3.0,0.0,555.135733,1232.870547,1468.184069,1089.496435,13.519393,...,137.547188,145.687444,19.001082,6.042239,31.538067,34.080248,-0.008968,-0.005504,-0.017672,0
3604564,502.756750,16.0,63.0,3.0,0.0,554.678067,1229.818158,1459.263290,1084.383871,13.498591,...,136.681546,144.774412,18.987135,6.006830,31.808141,33.798049,-0.008968,-0.005504,-0.017672,0
3604565,502.770538,16.0,63.0,3.0,0.0,554.157953,1227.944149,1458.169108,1085.181024,13.476994,...,135.983569,144.034985,18.984340,6.002011,31.754596,33.824353,-0.008968,-0.005504,-0.017672,0
3604566,502.800780,16.0,63.0,3.0,0.0,553.690880,1224.529086,1448.463089,1079.716205,13.456842,...,135.009254,143.009394,18.971948,5.944589,32.050241,33.521120,-0.008968,-0.005504,-0.017672,0


In [41]:
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import StandardScaler

# Assuming df is your Pandas DataFrame
# Separate the 'RUL' column from the rest of the data
rul_values= train['RUL'].values
features = train.drop(columns=['RUL'])

features.columns = features.columns.astype(str)
# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to your features and transform them
scaled_features = scaler.fit_transform(features)

# Convert the scaled features back to a DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# Concatenate the scaled features with the 'RUL' column
scaled_df['RUL'] = rul_values

# Now, scaled_df contains the scaled features with the 'RUL' column intact
train=scaled_df
train=train.drop(columns=["unit","Fc"])

In [40]:
rul_values= test['RUL'].values
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import StandardScaler


rul_values= test['RUL'].values
features = test

features.columns = features.columns.astype(str)
# Initialize StandardScaler
scaler = StandardScaler()

# Fit the scaler to your features and transform them
scaled_features = scaler.fit_transform(features)

# Convert the scaled features back to a DataFrame
scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# Concatenate the scaled features with the 'RUL' column


# Now, scaled_df contains the scaled features with the 'RUL' column intact
test=scaled_df
# test=test.drop(columns=["unit","Fc","LPC_flow_mod"])
test=test.drop(columns=["unit","Fc"])
test["RUL"]=rul_values

In [42]:
test

,T2,cycle,hs,T24,T30,T48,T50,P15,P2,P21,...,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod,RUL
0,1.293728,-1.660380,1.376490,2.011258,1.948641,1.668739,2.096150,1.908224,1.463237,1.908224,...,2.360849,2.360619,-1.710277,1.859079,-1.254824,1.426255,0.816670,0.775131,0.817560,58
1,1.294732,-1.660380,1.376490,2.011886,1.948098,1.668435,2.095330,1.908156,1.463442,1.908156,...,2.359885,2.359658,-1.705908,1.852450,-1.254025,1.426277,0.816670,0.775131,0.817560,58
2,1.295365,-1.660380,1.376490,2.012488,1.948232,1.668191,2.094203,1.907752,1.463137,1.907752,...,2.359326,2.359094,-1.701208,1.850062,-1.253366,1.425632,0.816670,0.775131,0.817560,58
3,1.292609,-1.660380,1.376490,2.010080,1.946527,1.666185,2.091261,1.905243,1.460713,1.905243,...,2.357094,2.356873,-1.703630,1.849680,-1.251590,1.423463,0.816670,0.775131,0.817560,58
4,1.289338,-1.660380,1.376490,2.006616,1.944606,1.665013,2.089807,1.901569,1.457381,1.901569,...,2.353819,2.353597,-1.705056,1.852918,-1.252090,1.422935,0.816670,0.775131,0.817560,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1253738,0.914691,1.807534,-0.726485,-1.148779,-2.142101,-2.182194,-0.780814,0.500944,1.187487,0.500944,...,-0.848088,-0.847448,-1.901480,-1.354496,1.999531,-2.292211,-2.782562,-4.232444,-2.252511,0
1253739,0.914381,1.807534,-0.726485,-1.152261,-2.140130,-2.168420,-0.757906,0.500261,1.187660,0.500261,...,-0.848099,-0.847486,-1.908423,-1.338093,1.967624,-2.266764,-2.782562,-4.232444,-2.252511,0
1253740,0.914884,1.807534,-0.726485,-1.151115,-2.139523,-2.171563,-0.763331,0.501409,1.188499,0.501409,...,-0.846710,-0.846101,-1.913105,-1.330920,1.975466,-2.273735,-2.782562,-4.232444,-2.252511,0
1253741,0.915087,1.807534,-0.726485,-1.150875,-2.139141,-2.170307,-0.760892,0.502043,1.189094,0.502043,...,-0.846265,-0.845657,-1.919067,-1.331206,1.973386,-2.271444,-2.782562,-4.232444,-2.252511,0


In [15]:
train=train.drop(columns=["cycle"])


KeyError: "['cycle'] not found in axis"

In [8]:
train

,T2,cycle,hs,T24,T30,T48,T50,P15,P2,P21,...,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod,RUL
0,1.677064,-1.660443,1.712388,2.414171,2.301052,1.944710,2.504000,2.348882,1.884787,2.348882,...,2.848425,2.848011,-1.640899,1.752864,-1.338155,1.606735,0.836907,1.082344,0.787664,62
1,1.675780,-1.660443,1.712388,2.418316,2.300973,1.935317,2.486674,2.348973,1.883031,2.348973,...,2.848908,2.848598,-1.645862,1.737838,-1.319097,1.589435,0.836907,1.082344,0.787664,62
2,1.674779,-1.660443,1.712388,2.416417,2.299067,1.935597,2.487492,2.346765,1.881497,2.346765,...,2.845799,2.845497,-1.643448,1.729597,-1.321345,1.591717,0.836907,1.082344,0.787664,62
3,1.673112,-1.660443,1.712388,2.414662,2.297930,1.934323,2.485543,2.344676,1.879576,2.344676,...,2.843880,2.843575,-1.643350,1.731392,-1.320667,1.590309,0.836907,1.082344,0.787664,62
4,1.672658,-1.660443,1.712388,2.413871,2.297068,1.933950,2.484877,2.343363,1.878545,2.343363,...,2.842197,2.841890,-1.641205,1.729520,-1.320742,1.590410,0.836907,1.082344,0.787664,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765290,1.620908,1.736893,-0.583980,-0.197469,-1.534778,-1.767321,-0.314074,1.217501,1.839586,1.217501,...,-0.166002,-0.164897,-0.157867,-2.198148,1.786860,-2.030510,-2.835198,-2.740140,-3.164987,0
765291,1.620771,1.736893,-0.583980,-0.222541,-1.589782,-1.858529,-0.413319,1.208030,1.839909,1.208030,...,-0.197986,-0.196812,-0.167077,-2.235115,1.920133,-2.157435,-2.835198,-2.740140,-3.164987,0
765292,1.621576,1.736893,-0.583980,-0.251033,-1.623552,-1.869716,-0.397844,1.198196,1.841200,1.198196,...,-0.223775,-0.222659,-0.168923,-2.240146,1.893710,-2.145605,-2.835198,-2.740140,-3.164987,0
765293,1.623341,1.736893,-0.583980,-0.276621,-1.685092,-1.968952,-0.503927,1.189021,1.843107,1.189021,...,-0.259774,-0.258508,-0.177105,-2.300093,2.039602,-2.281990,-2.835198,-2.740140,-3.164987,0


In [15]:
import numpy as np

# Assuming `data` is your multivariate time series dataframe
correlations = train.drop(columns=["RUL"]).corr()
threshold = 0.5  # Adjust based on your requirements
adjacency_matrix = (correlations.abs() > threshold).astype(int)
adjacency_matrix

,T2,cycle,hs,T24,T30,T48,T50,P15,P2,P21,...,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod
T2,1,0,0,1,0,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
cycle,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
hs,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T24,1,0,0,1,1,1,1,1,1,1,...,1,1,1,0,0,1,1,0,0,0
T30,0,0,0,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,0,0,0
T48,0,0,0,1,1,1,1,0,0,0,...,1,1,1,1,1,1,1,0,0,0
T50,1,0,0,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
P15,1,0,0,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
P2,1,0,0,1,0,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
P21,1,0,0,1,1,0,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0


In [10]:
adjacency_matrix.at["SmLPC","HPT_eff_mod"]=1
adjacency_matrix.at["HPT_eff_mod","SmLPC"]=1

In [43]:
adjacency_matrix

,T2,hs,T24,T30,T48,T50,P15,P2,P21,P24,...,W32,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod
T2,1,0,1,0,0,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
hs,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
T24,1,0,1,1,1,1,1,1,1,1,...,1,1,1,0,0,1,1,0,0,0
T30,0,0,1,1,1,1,1,0,1,1,...,1,1,1,1,1,1,1,0,0,0
T48,0,0,1,1,1,1,0,0,0,0,...,1,1,1,1,1,1,1,0,0,0
T50,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0
P15,1,0,1,1,0,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
P2,1,0,1,0,0,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
P21,1,0,1,1,0,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
P24,1,0,1,1,0,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0


In [16]:
adjacency_matrix=adjacency_matrix.drop(columns=["cycle"])
adjacency_matrix=adjacency_matrix.drop(index="cycle")

In [17]:
assert adjacency_matrix.shape == (33, 33), "Adjacency matrix should be of shape (33, 33)"

In [44]:
# i have to tranform from adjancey format to edge_index format  
import numpy as np
import torch

# Example adjacency matrix
adj_matrix =adjacency_matrix

# Find the indices where there are edges (non-zero values)
src, dst = np.where(adj_matrix == 1)

# Stack the source and destination arrays to form the edge_index
edge_index = np.vstack((src, dst))

# Convert edge_index to a PyTorch tensor
edge_index = torch.tensor(edge_index, dtype=torch.long)

print("Adjacency Matrix:")
print(adj_matrix)
print("\nEdge Index:")
print(edge_index)


Adjacency Matrix:
              T2  hs  T24  T30  T48  T50  P15  P2  P21  P24  ...  W32  W48  \
T2             1   0    1    0    0    1    1   1    1    1  ...    1    1   
hs             0   1    0    0    0    0    0   0    0    0  ...    0    0   
T24            1   0    1    1    1    1    1   1    1    1  ...    1    1   
T30            0   0    1    1    1    1    1   0    1    1  ...    1    1   
T48            0   0    1    1    1    1    0   0    0    0  ...    1    1   
T50            1   0    1    1    1    1    1   1    1    1  ...    1    1   
P15            1   0    1    1    0    1    1   1    1    1  ...    1    1   
P2             1   0    1    0    0    1    1   1    1    1  ...    1    1   
P21            1   0    1    1    0    1    1   1    1    1  ...    1    1   
P24            1   0    1    1    0    1    1   1    1    1  ...    1    1   
Ps30           1   0    1    1    1    1    1   1    1    1  ...    1    1   
P40            1   0    1    1    1    1    1 

In [21]:
train

,T2,cycle,hs,T24,T30,T48,T50,P15,P2,P21,...,W48,W50,SmFan,SmLPC,SmHPC,phi,HPT_eff_mod,LPT_eff_mod,LPT_flow_mod,RUL
0,1.677064,-1.660443,1.712388,2.414171,2.301052,1.944710,2.504000,2.348882,1.884787,2.348882,...,2.848425,2.848011,-1.640899,1.752864,-1.338155,1.606735,0.836907,1.082344,0.787664,62
1,1.675780,-1.660443,1.712388,2.418316,2.300973,1.935317,2.486674,2.348973,1.883031,2.348973,...,2.848908,2.848598,-1.645862,1.737838,-1.319097,1.589435,0.836907,1.082344,0.787664,62
2,1.674779,-1.660443,1.712388,2.416417,2.299067,1.935597,2.487492,2.346765,1.881497,2.346765,...,2.845799,2.845497,-1.643448,1.729597,-1.321345,1.591717,0.836907,1.082344,0.787664,62
3,1.673112,-1.660443,1.712388,2.414662,2.297930,1.934323,2.485543,2.344676,1.879576,2.344676,...,2.843880,2.843575,-1.643350,1.731392,-1.320667,1.590309,0.836907,1.082344,0.787664,62
4,1.672658,-1.660443,1.712388,2.413871,2.297068,1.933950,2.484877,2.343363,1.878545,2.343363,...,2.842197,2.841890,-1.641205,1.729520,-1.320742,1.590410,0.836907,1.082344,0.787664,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
765290,1.620908,1.736893,-0.583980,-0.197469,-1.534778,-1.767321,-0.314074,1.217501,1.839586,1.217501,...,-0.166002,-0.164897,-0.157867,-2.198148,1.786860,-2.030510,-2.835198,-2.740140,-3.164987,0
765291,1.620771,1.736893,-0.583980,-0.222541,-1.589782,-1.858529,-0.413319,1.208030,1.839909,1.208030,...,-0.197986,-0.196812,-0.167077,-2.235115,1.920133,-2.157435,-2.835198,-2.740140,-3.164987,0
765292,1.621576,1.736893,-0.583980,-0.251033,-1.623552,-1.869716,-0.397844,1.198196,1.841200,1.198196,...,-0.223775,-0.222659,-0.168923,-2.240146,1.893710,-2.145605,-2.835198,-2.740140,-3.164987,0
765293,1.623341,1.736893,-0.583980,-0.276621,-1.685092,-1.968952,-0.503927,1.189021,1.843107,1.189021,...,-0.259774,-0.258508,-0.177105,-2.300093,2.039602,-2.281990,-2.835198,-2.740140,-3.164987,0


In [13]:
# import numpy as np
# import pandas as pd

# # Assuming `train` is your DataFrame with the data
# # and it has a "Cycle_ID" column to identify different cycles

# # Drop the "RUL" column and convert to numpy array for efficient processing
# train_data = train.drop(columns=["RUL"]).values

# # Initialize the list for feature matrices
# feature_matrices = []

# # Group by Cycle_ID and apply sliding window within each group
# cycle_ids = train["cycle"].unique()

# for cycle_id in cycle_ids:
#     # Get data for the current cycle
#     cycle_data = train[train["cycle"] == cycle_id].drop(columns=["RUL", "cycle"]).values
    
#     num_rows = cycle_data.shape[0]
#     window_size = 10

#     # Create feature matrices for the current cycle
#     for i in range(num_rows - window_size + 1):
#         feature_matrix = cycle_data[i:i + window_size].T  # Transpose to get each feature as a row
#         feature_matrices.append(feature_matrix)

# # Convert to numpy array for efficient processing later
# feature_matrices = np.array(feature_matrices)


# 

In [22]:
import numpy as np
import pandas as pd

# Assuming `train` is your DataFrame with the data
# and it has a "Cycle_ID" column to identify different cycles

# Drop the "RUL" column and convert to numpy array for efficient processing
train_data = train.drop(columns=["RUL"]).values

# Initialize the lists for feature matrices and target RUL values
feature_matrices = []
target_rul = []

# Group by Cycle_ID and apply sliding window within each group
cycle_ids = train["cycle"].unique()

for cycle_id in cycle_ids:
    # Get data for the current cycle
    cycle_data = train[train["cycle"] == cycle_id]
    cycle_features = cycle_data.drop(columns=["RUL", "cycle"]).values
    cycle_rul = cycle_data["RUL"].values
    
    num_rows = cycle_features.shape[0]
    window_size = 10

    # Create feature matrices and corresponding RUL targets for the current cycle
    for i in range(num_rows - window_size + 1):
        feature_matrix = cycle_features[i:i + window_size].T  # Transpose to get each feature as a row
        feature_matrices.append(feature_matrix)
        
        # Extract the RUL value at the end of the window
        target_rul.append(cycle_rul[i + window_size - 1])

# Convert to numpy arrays for efficient processing later
feature_matrices = np.array(feature_matrices)
target_rul = np.array(target_rul)

# Print shapes for verification
print(f"Feature matrices shape: {feature_matrices.shape}")
print(f"Target RUL shape: {target_rul.shape}")


Feature matrices shape: (764728, 33, 10)
Target RUL shape: (764728,)


In [23]:
feature_tensor_matrices=torch.tensor(feature_matrices,dtype=torch.float)
feature_tensor_matrices[0]

tensor([[ 1.6771,  1.6758,  1.6748,  1.6731,  1.6727,  1.6722,  1.6704,  1.6708,
          1.6697,  1.6689],
        [ 1.7124,  1.7124,  1.7124,  1.7124,  1.7124,  1.7124,  1.7124,  1.7124,
          1.7124,  1.7124],
        [ 2.4142,  2.4183,  2.4164,  2.4147,  2.4139,  2.4133,  2.4115,  2.4070,
          2.4055,  2.4101],
        [ 2.3011,  2.3010,  2.2991,  2.2979,  2.2971,  2.2964,  2.2951,  2.2826,
          2.2932,  2.2933],
        [ 1.9447,  1.9353,  1.9356,  1.9343,  1.9339,  1.9333,  1.9319,  1.9117,
          1.9386,  1.9287],
        [ 2.5040,  2.4867,  2.4875,  2.4855,  2.4849,  2.4835,  2.4813,  2.4560,
          2.4932,  2.4751],
        [ 2.3489,  2.3490,  2.3468,  2.3447,  2.3434,  2.3422,  2.3398,  2.3366,
          2.3352,  2.3360],
        [ 1.8848,  1.8830,  1.8815,  1.8796,  1.8785,  1.8775,  1.8754,  1.8748,
          1.8731,  1.8719],
        [ 2.3489,  2.3490,  2.3468,  2.3447,  2.3434,  2.3422,  2.3398,  2.3366,
          2.3352,  2.3360],
        [ 2.5559,  

In [56]:
import torch
from torch_geometric.data import Data

dataset = []
for i in range(len(feature_tensor_matrices)):
    # Convert feature tensors to float32
    x = feature_tensor_matrices[i].float()
    # Ensure edge_index is long
    edge_index = edge_index.long()
    # Convert y values to float32
    y = torch.tensor([target_rul[i]], dtype=torch.float32)
    
    data = Data(x=x, edge_index=edge_index, y=y)
    dataset.append(data)

In [57]:
edge_index

tensor([[ 0,  0,  0,  ..., 32, 32, 32],
        [ 0,  2,  5,  ..., 30, 31, 32]])

In [58]:
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data

class CustomGraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        return self.data_list[index]

# Example usage:
# Assuming data_objects is a list of your PyTorch Geometric Data objects

# Create the custom dataset
custom_dataset = CustomGraphDataset(dataset)


# Check the length of the dataset
print(len(custom_dataset))

# Access a specific graph from the dataset
graph_data = custom_dataset[0]
print(graph_data)


764728
Data(x=[33, 10], edge_index=[2, 671], y=[1])


In [26]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool

class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)  # Global mean pooling
        x = self.fc(x)
                
        
        return x



In [62]:
feature_tensor_matrices[0].shape[1]

10

In [30]:
import torch
from torch_geometric.loader import DataLoader

# Assuming custom_dataset is your instance of CustomGraphDataset
dataloader = DataLoader(custom_dataset, batch_size=32, shuffle=True)

# Model, loss, and optimizer
model = GCN(input_dim=10, hidden_dim=16, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

# # Training loop
model.train()
for epoch in range(50):
    total_loss = 0
    for data in dataloader:
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(dataloader)}')


In [45]:
import torch
import pickle

model = torch.load('GCN_model.pth')
model.eval()


GCN(
  (conv1): GCNConv(10, 16)
  (conv2): GCNConv(16, 16)
  (fc): Linear(in_features=16, out_features=1, bias=True)
)

In [46]:
import numpy as np
import pandas as pd

# Assuming `train` is your DataFrame with the data
# and it has a "Cycle_ID" column to identify different cycles

# Drop the "RUL" column and convert to numpy array for efficient processing
train_data = test.drop(columns=["RUL"]).values

# Initialize the lists for feature matrices and target RUL values
feature_matrices = []
target_rul = []

# Group by Cycle_ID and apply sliding window within each group
cycle_ids = test["cycle"].unique()

for cycle_id in cycle_ids:
    # Get data for the current cycle
    cycle_data = test[test["cycle"] == cycle_id]
    cycle_features = cycle_data.drop(columns=["RUL", "cycle"]).values
    cycle_rul = cycle_data["RUL"].values
    
    num_rows = cycle_features.shape[0]
    window_size = 10

    # Create feature matrices and corresponding RUL targets for the current cycle
    for i in range(num_rows - window_size + 1):
        feature_matrix = cycle_features[i:i + window_size].T  # Transpose to get each feature as a row
        feature_matrices.append(feature_matrix)
        
        # Extract the RUL value at the end of the window
        target_rul.append(cycle_rul[i + window_size - 1])

# Convert to numpy arrays for efficient processing later
feature_matrices = np.array(feature_matrices)
target_rul = np.array(target_rul)

# Print shapes for verification
print(f"Feature matrices shape: {feature_matrices.shape}")
print(f"Target RUL shape: {target_rul.shape}")


Feature matrices shape: (1253059, 33, 10)
Target RUL shape: (1253059,)


In [ ]:
import torch
import pickle

model = torch.load('GCN_model.pth')
model.eval()


In [47]:
feature_tensor_matrices=torch.tensor(feature_matrices,dtype=torch.float)
feature_tensor_matrices[0]

tensor([[ 1.2937,  1.2947,  1.2954,  1.2926,  1.2893,  1.2874,  1.2868,  1.2862,
          1.2868,  1.2861],
        [ 1.3765,  1.3765,  1.3765,  1.3765,  1.3765,  1.3765,  1.3765,  1.3765,
          1.3765,  1.3765],
        [ 2.0113,  2.0119,  2.0125,  2.0101,  2.0066,  2.0043,  2.0035,  2.0028,
          2.0032,  2.0026],
        [ 1.9486,  1.9481,  1.9482,  1.9465,  1.9446,  1.9429,  1.9423,  1.9416,
          1.9415,  1.9408],
        [ 1.6687,  1.6684,  1.6682,  1.6662,  1.6650,  1.6640,  1.6638,  1.6631,
          1.6631,  1.6621],
        [ 2.0962,  2.0953,  2.0942,  2.0913,  2.0898,  2.0886,  2.0882,  2.0870,
          2.0864,  2.0844],
        [ 1.9082,  1.9082,  1.9078,  1.9052,  1.9016,  1.8992,  1.8980,  1.8968,
          1.8963,  1.8950],
        [ 1.4632,  1.4634,  1.4631,  1.4607,  1.4574,  1.4553,  1.4544,  1.4533,
          1.4531,  1.4519],
        [ 1.9082,  1.9082,  1.9078,  1.9052,  1.9016,  1.8992,  1.8980,  1.8968,
          1.8963,  1.8950],
        [ 2.0983,  

In [48]:
import torch
from torch_geometric.data import Data

dataset = []
for i in range(len(feature_tensor_matrices)):
    # Convert feature tensors to float32
    x = feature_tensor_matrices[i].float()
    # Ensure edge_index is long
    edge_index = edge_index.long()
    # Convert y values to float32
    y = torch.tensor([target_rul[i]], dtype=torch.float32)
    
    data = Data(x=x, edge_index=edge_index, y=y)
    dataset.append(data)

In [49]:
import torch
from torch.utils.data import Dataset
from torch_geometric.data import Data

class CustomGraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list
    
    def __len__(self):
        return len(self.data_list)
    
    def __getitem__(self, index):
        return self.data_list[index]

# Example usage:
# Assuming data_objects is a list of your PyTorch Geometric Data objects

# Create the custom dataset
custom_dataset = CustomGraphDataset(dataset)


# Check the length of the dataset
print(len(custom_dataset))

# Access a specific graph from the dataset
graph_data = custom_dataset[0]
print(graph_data)


1253059
Data(x=[33, 10], edge_index=[2, 671], y=[1])


In [51]:
import torch
import numpy as np
from torch_geometric.loader import DataLoader
from sklearn.metrics import mean_squared_error

# Assuming CustomGraphDataset and test_dataset are defined
test_dataloader = DataLoader(custom_dataset, batch_size=32, shuffle=False)

# Load the trained model
# model = GCN(input_dim=10, hidden_dim=16, output_dim=1)
# model.load_state_dict(torch.load('GCN_model.pth'))
# model.eval()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

all_predictions = []
all_labels = []

with torch.no_grad():
    for data in test_dataloader:
        data = data.to(device)
        out = model(data)
        all_predictions.append(out.squeeze().cpu().numpy())
        all_labels.append(data.y.cpu().numpy())

all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(all_labels, all_predictions))
print(f'RMSE: {rmse}')


RMSE: 8.62903881072998
